In [1]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.io import loadmat
import matplotlib.pyplot as plt

In [2]:
def updateWLRAgrad(X, W, v, u, step=None):
    R = u.dot(v.T) - X  # residual
    WR = W.multiply(R)  # masked residual
    gradu = WR.dot(v)   # gradient at u

    if step is None or not step:  # Check if step is empty or None
        step = 2 * np.linalg.norm(u) / np.linalg.norm(gradu)

    # print(step)
    e0 = np.sum(WR.power(2))
    e1 = float('inf')
    while e1 > e0:
        unew = u - step * gradu
        step /= 1.5 #( 1/1.5 = \gamma)
        e1 = np.sum((W.multiply(X - unew.dot(v.T))).power(2))
    u = unew
    step *= 4       #( 4 = \beta)
    return u, step, e1

In [4]:
# Load data matrix
data = loadmat('Data/matlab/inputX.mat')  # Load the .mat file
X = data['X']  # Assuming the matrix is stored under the key 'X'
X = csr_matrix(X)  # Convert to sparse matrix
m, n = X.shape
numX = X.count_nonzero()

# Construct binary weight matrix
W = X.copy()
W.data = np.ones_like(W.data)

# Initialization: Compute a rank-one (approximate) factorization
# Init.1: random in the interval [1,5]
# u = np.random.rand(m, 5)
# v = np.random.rand(n, 5) * 5

# Init.2: average per row
rank = 5
v = np.ones((n, rank))
u = np.random.rand(m, rank)
for i in range(m):
    non_zero_indices = X[i].nonzero()[1]
    if len(non_zero_indices) > 0:
        u[i] = np.array([X[i, non_zero_indices].mean() for _ in range(len(u[i]))])

# Estimate the parameters of the model: Optimize u and v alternatively via gradient descent
k = 1
max_k = 100
uprec = u
stepu = []
stepv = []
e = []
while k <= max_k and (k <= 2 or (e[-2] - e[-1]) / e[-2] > 1e-6):  # Stopping criterion
    # Update u
    u, stepu, eu = updateWLRAgrad(X, W, v, u, stepu)
    # Update v
    v, stepv, ev = updateWLRAgrad(X.T, W.T, u, v, stepv)
    e.extend([np.sqrt(eu / numX), np.sqrt(ev / numX)])
    print(f"Iteration {k}, RMSE: {e[-1]}")
    k += 1

# Display the evolution of the RMSE
plt.figure()
plt.xlabel('Iteration')
plt.ylabel('Observed RMSE')
plt.yscale('log')
plt.plot(e, 'bo--')
plt.show()

Iteration 1, RMSE: 11.019023841459507
Iteration 2, RMSE: 7.3261453864265045
Iteration 3, RMSE: 4.206875720840425


KeyboardInterrupt: 

In [7]:
# Construct the prediction in a csv file
data_eval = loadmat('Data/matlab/inputEval.mat')
Eval = data_eval['Eval']
numpred = Eval.shape[0]
predic = np.zeros((numpred,2))
# predic[:,0] = np.linspace(1, numpred, numpred, endpoint=True).round().astype(int)
for i in range(numpred):
    predic[i,0] = int(i + 1)
    predic[i,1] = np.dot(u[Eval[i,0]-1], v[Eval[i,1]-1])
    # Make sure the prediction is in [1,5]
    predic[i,1] = max(1, predic[i,1])
    predic[i,1] = min(5, predic[i,1])
np.savetxt('sol_rank_one.csv', predic, delimiter=',', fmt=["%d"] + ["%.4f"], header="ID,Rating",comments='')